<a href="https://colab.research.google.com/github/Ellen-Tuane/camera_calibration/blob/main/Camera_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install opencv-contrib-python==3.4.13.47 --force-reinstall

     |████████████████████████████████| 53.7 MB 186 kB/s 
     |████████████████████████████████| 15.7 MB 31.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!mkdir images

In [4]:
!cp -a /content/drive/MyDrive/yolov4/calibration/.  /content/images/

In [4]:
%cd /content/images/calibration

/content/images/calibration


In [6]:
!# Import required modules
import cv2
import numpy as np
import os
import glob


# Define the dimensions of checkerboard
CHECKERBOARD = (9, 6)


# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# Vector for 3D points
threedpoints = []

# Vector for 2D points
twodpoints = []


# 3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0]
                    * CHECKERBOARD[1],
                    3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
                            0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None


# Extracting path of individual image stored
# in a given directory. Since no path is
# specified, it will take current directory
# jpg files alone
images = glob.glob('*.HEIC')

print(len(images))

for filename in images:
    image = cv2.imread(filename)
    grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(grayColor, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    # If desired number of corners can be detected then,
    # refine the pixel coordinates and display
    # them on the images of checker board
    if not ret:
        pass
        print('erro', filename)
    else:
        threedpoints.append(objectp3d)

        # Refining pixel coordinates
        # for given 2d points.
        corners2 = cv2.cornerSubPix(grayColor, corners, (11, 11), (-1, -1), criteria)

        twodpoints.append(corners2)

        # Draw and display the corners
        image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)
        img = cv2.resize(image, (960, 540))
        # cv2.imshow(f"{filename}", img)
        save_path = '/content/drive/MyDrive/yolov4/calibration_output '
        cv2.imwrite(f"{save_path}/{filename}.jpg", img)
        # cv2.waitKey(10)
        print(filename)

# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(threedpoints, twodpoints, grayColor.shape[::-1], None, None)


# Displaying required output
print(" Camera matrix:")
print(matrix)

print("\n Distortion coefficient:")
print(distortion)

print("\n Rotation Vectors:")
print(r_vecs)

print("\n Translation Vectors:")
print(t_vecs)

72
IMG_6927.HEIC
IMG_6931.HEIC
IMG_6911.HEIC
IMG_6922.HEIC
IMG_6951.HEIC
IMG_6902.HEIC
IMG_6948.HEIC
IMG_6943.HEIC
IMG_6898.HEIC
IMG_6890.HEIC
IMG_6932.HEIC
IMG_6934.HEIC
IMG_6930.HEIC
IMG_6916.HEIC
IMG_6887.HEIC
IMG_6903.HEIC
IMG_6937.HEIC
IMG_6952.HEIC
IMG_6914.HEIC
IMG_6941.HEIC
IMG_6925.HEIC
IMG_6944.HEIC
IMG_6915.HEIC
IMG_6888.HEIC
IMG_6936.HEIC
IMG_6892.HEIC
IMG_6907.HEIC
IMG_6904.HEIC
IMG_6874.HEIC
IMG_6884.HEIC
IMG_6908.HEIC
IMG_6881.HEIC
IMG_6909.HEIC
IMG_6912.HEIC
IMG_6879.HEIC
IMG_6929.HEIC
IMG_6935.HEIC
IMG_6885.HEIC
IMG_6893.HEIC
IMG_6899.HEIC
IMG_6897.HEIC
IMG_6891.HEIC
IMG_6894.HEIC
IMG_6920.HEIC
IMG_6942.HEIC
IMG_6924.HEIC
IMG_6947.HEIC
IMG_6917.HEIC
IMG_6882.HEIC
IMG_6913.HEIC
IMG_6901.HEIC
IMG_6940.HEIC
IMG_6926.HEIC
IMG_6900.HEIC
IMG_6949.HEIC
IMG_6883.HEIC
IMG_6886.HEIC
IMG_6910.HEIC
IMG_6933.HEIC
IMG_6895.HEIC
IMG_6939.HEIC
IMG_6921.HEIC
IMG_6919.HEIC
IMG_6945.HEIC
IMG_6918.HEIC
IMG_6928.HEIC
IMG_6906.HEIC
IMG_6905.HEIC
IMG_6896.HEIC
IMG_6946.HEIC
IMG_6889.HEIC
IMG